In [ ]:
import ipywidgets as widgets
from IPython.display import display
import matplotlib.pyplot as plt
import random

%matplotlib inline

class Load:
    # For now, sinusoidal time-varying server load model with peak in the afternoon.
    # Load between 0 and 1, in 2-hour intervals starting at 0:00-1:59.
    load_by_hours = [
        0.4, 0.4, 0.4, 0.4, 0.6, 0.8, 1.0, 1.0, 1.0, 0.8, 0.6, 0.6
    ]

    @staticmethod
    def at_hour(hour):
        assert 0 <= hour < 24
        return max(0, min(Load.load_by_hours[hour // 2] + random.uniform(-0.1, 0.1), 1))


class Emissions:
    # Average direct (not including LCA) emissions factors in g CO2e per kWh
    # for electricity consumed in Belgium for each pair of hours starting
    # with 0:00-1:59, based on hourly data for 2023 from https://electricitymaps.com/.
    gco2_per_kwh_by_hours = [
        120, 116, 124, 128, 113, 100, 97, 106, 135, 150, 148, 129
    ]

    @staticmethod
    def at_hour(hour):
        return Emissions.gco2_per_kwh_by_hours[hour // 2]

    @staticmethod
    def average():
        return sum(Emissions.gco2_per_kwh_by_hours) / 12

class CloudDataCenter:
    def __init__(self, base_it_load_kw, pue):
        self.base_it_load_kw = base_it_load_kw
        self.pue = pue

    def hourly_energy_and_emissions(self, start_hour, end_hour):
        assert 0 <= start_hour < 24 and 0 <= end_hour < 24
        hourly_data = []

        for hour in range(start_hour, end_hour + 1):
            it_kwh = self.base_it_load_kw * Load.at_hour(hour)
            total_kwh = it_kwh * self.pue
            emissions_per_kwh = Emissions.at_hour(hour) / 1000  # Convert g to kg
            it_emissions = it_kwh * emissions_per_kwh
            total_emissions = total_kwh * emissions_per_kwh

            hourly_data.append({
                'hour': hour,
                'it_kwh': it_kwh,
                'total_kwh': total_kwh,
                'it_emissions': it_emissions,
                'total_emissions': total_emissions,
            })

        return hourly_data

    def summary(self, start_hour, end_hour):
        data = self.hourly_energy_and_emissions(start_hour, end_hour)
        total_it_kwh = sum(d['it_kwh'] for d in data)
        total_kwh = sum(d['total_kwh'] for d in data)
        total_emissions = sum(d['total_emissions'] for d in data)
        return {
            'total_it_kwh': total_it_kwh,
            'total_kwh': total_kwh,
            'total_emissions': total_emissions,
            'pue': self.pue,
            'cue': total_emissions / total_it_kwh if total_it_kwh > 0 else float('inf'),
        }

style = {'description_width': 'initial'}
it_load_slider = widgets.FloatSlider(value=500, min=10, max=5000, step=10, description='IT Load (kW):', style=style)
pue_slider = widgets.FloatSlider(value=1.2, min=1.1, max=3.0, step=0.01, description='PUE:', style=style)
start_hour_slider = widgets.IntSlider(value=0, min=0, max=23, step=1, description='Start Hour:', style=style)
end_hour_slider = widgets.IntSlider(value=23, min=0, max=23, step=1, description='End Hour:', style=style)
graphs = widgets.Output(layout={'border': '1px solid black', 'width': '99%'})
info_text = widgets.Textarea(value='', layout=widgets.Layout(width='99%', height='120px', overflow='hidden'))
update_button = widgets.Button(description='Update',button_style='success')
fig, ax1, ax2 = None, None, None


it_box = widgets.VBox([it_load_slider, pue_slider])
time_box = widgets.HBox([start_hour_slider, end_hour_slider])
main_box = widgets.VBox([it_box, time_box, update_button, graphs, info_text])
display(main_box)

def update(b):
    plt.ioff()
    data_center = CloudDataCenter(it_load_slider.value, pue_slider.value)
    start_hour, end_hour = start_hour_slider.value, end_hour_slider.value
    hourly_data = data_center.hourly_energy_and_emissions(start_hour, end_hour)

    hours = [data['hour'] for data in hourly_data]
    it_kwh = [data['it_kwh'] for data in hourly_data]
    total_kwh = [data['total_kwh'] for data in hourly_data]
    total_emissions = [data['total_emissions'] for data in hourly_data]

    global fig, ax1, ax2
    if fig is None or ax1 is None or ax2 is None:
        fig, (ax1, ax2) = plt.subplots(nrows=1, ncols=2, figsize=(10, 3))
    ax1.clear()
    ax1.plot(hours, it_kwh, label='IT Load', linestyle='--')
    ax1.plot(hours, total_kwh, label='Total Load', linestyle='-')
    ax1.set_title("Hourly Energy Usage")
    ax1.set_xlabel("Hour")
    ax1.set_ylabel("Energy (kWh)")
    ax1.legend()
    ax1.grid(True)

    ax2.clear()
    ax2.plot(hours, total_emissions, label='Total Emissions', linestyle='-')
    ax2.set_title("Hourly Emissions")
    ax2.set_xlabel("Hour")
    ax2.set_ylabel("Emissions (kg CO2e)")
    ax2.legend()
    ax2.grid(True)
    fig.tight_layout()

    with graphs:
        display(fig, clear=True)

    summary = data_center.summary(start_hour, end_hour)
    info_text.value = (
        f"IT base load: {it_load_slider.value} kW\n"
        f"Total IT Energy (kWh): {summary['total_it_kwh']:.2f}\n"
        f"Total Energy (kWh): {summary['total_kwh']:.2f}\n"
        f"Total Emissions (kg CO2e): {summary['total_emissions']:.2f}\n"
        f"PUE: {summary['pue']:.2f}\n"
        f"CUE: {summary['cue']:.2f} kg CO2e/kWh"
    )

update_button.on_click(update)
update(None)
